# NIHCXR Synthetic Drift - Gaussian Shift

## Load Libraries

In [ ]:
from drift_detection.datasets import NIHCXRDataset
from drift_detection.drift_detector import (
    Detector,
    Experimenter,
    Reductor,
    SyntheticShiftApplicator,
    TSTester,
    plot_drift_samples_pval,
)
from drift_detection.drift_detector.utils import Loader

## Query Data

In [ ]:
# Load the dataset
dataset = NIHCXRDataset(cfg_path="../../datasets/configs/nihcxr.yaml")
dataset, metadata, metadata_mapping = dataset.get_data()

## Initalize Reductor, Tester & Detector

In [ ]:
reductor = Reductor(
    dr_method="TAE_txrv_CNN",
)

tester = TSTester(
    tester_method="mmd",
)


detector = Detector(
    reductor=reductor,
    tester=tester,
)
with Loader("Initializing the detector..."):
    detector.fit(dataset, progress=False)

## Setup Baseline Experiment

In [ ]:
baseline_experiment = Experimenter(
    "sensitivity_test",
    detector=detector,
)

## Setup Drift Experiment (Categorical Shift)

In [ ]:
shiftapplicator = SyntheticShiftApplicator(
    shift_type="gn_shift",
)

drift_experiment = Experimenter(
    "sensitivity_test",
    detector=detector,
    shiftapplicator=shiftapplicator,
)

## Run Experiments

In [ ]:
baseline_results = baseline_experiment.run(dataset, metadata, metadata_mapping)
drift_results = drift_experiment.run(dataset, metadata, metadata_mapping)

## Gather Results

In [ ]:
results = {"baseline": baseline_results, "gaussian_noise": drift_results}

## Plot Experimental Results

In [ ]:
plot_drift_samples_pval(results, 0.05)